In [2]:
from dataset import Raw_PhysionNet,PSD_PhysioNet
from torch.utils.data import  random_split,DataLoader
import torch
from classifier import Deep_Classifier
import os
import config_local
import torch.nn as nn
import itertools
import mne
from collections import defaultdict

In [4]:
os.listdir("../../../data/eeg-motor-movementimagery-dataset-1.0.0/files")

['S076',
 'S083',
 'S104',
 'S021',
 'S075',
 'S001',
 'S054',
 'S106',
 'S087',
 'S029',
 'S048',
 'S046',
 'S035',
 'S030',
 'S033',
 'S040',
 '64_channel_sharbrough-old.png',
 'S060',
 'S041',
 'S032',
 'S082',
 'S108',
 'S013',
 'S092',
 'S049',
 'S072',
 'S100',
 'S069',
 'S103',
 'S003',
 'S080',
 'S044',
 'S038',
 'S031',
 'S022',
 'S014',
 'S039',
 'S053',
 'S005',
 'S062',
 'S099',
 'S070',
 'S012',
 'S101',
 'S055',
 'SHA256SUMS.txt',
 'S063',
 'S004',
 'S107',
 'S037',
 'S025',
 'S034',
 'S018',
 'S098',
 'S011',
 'S066',
 'S050',
 'S109',
 'S047',
 'S015',
 'S009',
 'S024',
 'S091',
 'S093',
 'S095',
 'S102',
 'S089',
 'S026',
 'S071',
 'S057',
 'S084',
 'S017',
 '64_channel_sharbrough.pdf',
 'S051',
 'S079',
 'S010',
 'S088',
 'S077',
 'S020',
 'S059',
 'S073',
 'S002',
 'S078',
 'S008',
 'S056',
 'S086',
 'S027',
 'S061',
 'S064',
 'S028',
 'S007',
 'S097',
 'S068',
 'S016',
 'S085',
 'S019',
 'S096',
 'S052',
 'S074',
 'S045',
 'S105',
 'ANNOTATORS',
 'S065',
 'S090',
 '

In [18]:
file = "../../../data/eeg-motor-movementimagery-dataset-1.0.0/files/S001/S001R06.edf"
edf_data = mne.io.read_raw_edf(file, verbose=False,preload=True)


events, event_id = mne.events_from_annotations(edf_data)

tmin = float(os.getenv("EPOCH_MIN"))  # start of each epoch relative to the event
tmax = float(os.getenv("EPOCH_MAX"))   # end of each epoch relative to the event

# Create epochs from events
epochs = mne.Epochs(edf_data, events, {'T2': 3}, tmin, tmax,baseline=None,preload=True,verbose=False)        
epochs

Number of events,8
Events,T2: 8
Time range,0.000 – 2.000 s
Baseline,off


In [19]:
events

array([[    0,     0,     1],
       [  672,     0,     3],
       [ 1328,     0,     1],
       [ 2000,     0,     2],
       [ 2656,     0,     1],
       [ 3328,     0,     2],
       [ 3984,     0,     1],
       [ 4656,     0,     3],
       [ 5312,     0,     1],
       [ 5984,     0,     2],
       [ 6640,     0,     1],
       [ 7312,     0,     3],
       [ 7968,     0,     1],
       [ 8640,     0,     3],
       [ 9296,     0,     1],
       [ 9968,     0,     2],
       [10624,     0,     1],
       [11296,     0,     2],
       [11952,     0,     1],
       [12624,     0,     3],
       [13280,     0,     1],
       [13952,     0,     3],
       [14608,     0,     1],
       [15280,     0,     2],
       [15936,     0,     1],
       [16608,     0,     2],
       [17264,     0,     1],
       [17936,     0,     3],
       [18592,     0,     1],
       [19264,     0,     3]])

In [13]:
event_id

{'T0': 1, 'T1': 2, 'T2': 3}

Number of events,30
Events,T0: 15T1: 8T2: 7
Time range,0.000 – 2.000 s
Baseline,off


In [3]:
import sklearn.cluster._kmeans as kmeans

In [70]:
# 512, param 1
#     intra 0.9665
#     inter 0.6480

# 256, param 2
#     intra 0.9650
#     inter 0.6125

# 128, param 1
#     intra 0.9559
#     inter 0.5600


In [2]:
import numpy as np

In [3]:
def cosine_similarity_matrix(vectors):
    """
    Computes the cosine similarity matrix for a set of vectors.

    Parameters:
    vectors (torch.Tensor): A torch tensor of shape (n, 128) representing n vectors.

    Returns:
    torch.Tensor: An n x n matrix where element (i, j) represents the cosine similarity between vector i and vector j.
    """
    # Normalize the vectors
    norm_vectors = vectors / vectors.norm(dim=1, keepdim=True)
    
    # Compute the cosine similarity matrix
    similarity_matrix = torch.mm(norm_vectors, norm_vectors.T)
    
    return similarity_matrix

In [4]:

def generate_combinations(elements, length):
    """
    Generates all possible combinations of a given length from a list of elements.

    Parameters:
    elements (list): The list of elements to combine.
    length (int): The length of each combination.

    Returns:
    list: A list of lists, where each inner list is a combination.
    """
    return np.array(list(itertools.product(elements, repeat=length)))



In [5]:
def group_indices(tensor):
    groups = defaultdict(list)
    for idx, item in enumerate(tensor):
        groups[item.item()].append(idx)
    return list(groups.values())



In [6]:
model_cls = Deep_Classifier(num_classes=10)

In [7]:
model_cls.load_state_dict(torch.load("best_model.pth",map_location=torch.device('cpu')))

<All keys matched successfully>

In [8]:
raw_dataset = Raw_PhysionNet()
# psd_dataset = PSD_PhysioNet(raw_dataset=raw_dataset,freq_bin=bin_dict_extnd,duration=1)

.... found 1526 edf files ....


In [72]:
indicies = np.arange(0,100)

In [73]:
indicies_list = []
for class_idx in range(10):
    ind_cls = list(indicies + (class_idx)*(362*4))
    ind_cls = ind_cls[:20]
    indicies_list.append(ind_cls)

In [74]:
subj_data_x = [] # list to store data of all subjects

In [75]:
for cls_indicies in indicies_list:
    class_data_x = []
    # class_data_y = []
    for idx in cls_indicies:
        class_data_x.append(raw_dataset.__getitem__(idx)[0])
        # class_data_y.append(raw_dataset.__getitem__(idx)[1])
    subj_data_x.append(class_data_x)
    # subj_data_y.append(class_data_y)


In [76]:
subj_embeds = []

taking prediction

In [77]:
model_cls.eval()
for data in subj_data_x:
    input_tensor = torch.tensor(data)
    output = model_cls.embed(input_tensor.float(),3) # extracting 512 dim embedding for the batch
    subj_embeds.append(output)

In [78]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)


In [79]:
sims_inter = []

In [80]:
for j in range(10):
    for i in range(10):
        output = cos(subj_embeds[j],subj_embeds[i])
        out = torch.sum(output,dim=-1)/output.shape[0]
        del output
        sims_inter.append(out.data)

In [81]:
sum(sims_inter)/len(sims_inter)

tensor(0.5600)

In [82]:
sims_intra = []

In [83]:
for j in range(10):
    for i in range(10):
        output = cos(subj_embeds[j],subj_embeds[j][i])
        out = torch.sum(output,dim=-1)/output.shape[0]
        del output
        sims_intra.append(out.data)

In [84]:
sum(sims_intra)/len(sims_intra)

tensor(0.9559)

In [ ]:
n_cluster = 

In [ ]:
kmeans()

In [88]:
del subj_data_x
del subj_embeds

NameError: name 'subj_data_x' is not defined